# Tensorflow 簡易教學與實作

## 1. 介紹

Tensorflow 是由 Google 所發布的深度學習框架，讓使用者可以自行設計神經網路的架構，其優點在於在設計神經網路上非常靈活，使用者對於自己的神經路的運作可以有較全面的掌控，而缺點則是入門門檻相較於其他深度學習框架相對較高

## 2. 三種 Tensor

tensor 在數學上的意思是任意維度的矩陣，對於深度學習有初步的認識後就不難想像**神經網路就是由許多的 Tensor 所組成**，而 Tensorflow 提供了三種不同特性的 tensor 供使用者建構自己的神經網路

在介紹之前，先定義兩個名詞：

- tensor：任意維度的矩陣
- shape：tensor 在每一維度上的大小

### 2.1 tf.constant()

第一個 tensor 是 tf.constant() 所產生的 tensor，其他的特性為**一旦產生了以後便不能修改它的值**，用法可以參考以下程式碼：

In [1]:
import tensorflow as tf

# 使用 tf.constant() 產生 Tensor
a = tf.constant(0)

print(a)

Tensor("Const:0", shape=(), dtype=int32)


上面的程式碼產生了一個 tensor 被命名為 a 且其值為 0


接著如果想要產生內含矩陣的 tensor 可以這麼做：

In [2]:
# 產生內容為 1 維矩陣的 Tensor
b = tf.constant([1, 2, 3])
print(b)

# 產生內容為 2 維矩陣的 Tensor
c = tf.constant([[2, 4], [6, 8], [10, 12]])
print(c)

Tensor("Const_1:0", shape=(3,), dtype=int32)
Tensor("Const_2:0", shape=(3, 2), dtype=int32)


上面的程式碼產生了一個 1 維 tensor 一個 2 維 tensor，其中各自內容為：

$
b = 
\begin{bmatrix}
    1 & 2 & 3
\end{bmatrix}
$

$
c = 
\begin{bmatrix}
    2 & 4 \\
    6 & 8 \\
    10 & 12
\end{bmatrix}
$

### 2.2 tf.Variable()

tf.Variable() 的用法跟 tf.constant() 類似，它們之間的差在於 **tf.Variable() 產生出來的 tensor 其值可以被修改**，基本用法如下，與 tf.constant() 一樣

In [3]:
a = tf.Variable(0)
b = tf.Variable([1, 2, 3])
c = tf.Variable([[2, 4], [6, 8], [10, 12]])

print(a)
print(b)
print(c)

<tf.Variable 'Variable:0' shape=() dtype=int32_ref>
<tf.Variable 'Variable_1:0' shape=(3,) dtype=int32_ref>
<tf.Variable 'Variable_2:0' shape=(3, 2) dtype=int32_ref>


還有一些比較進階的用法，也是比較常用的用法，也就是將 tensor 裡面的值隨機指定：

In [4]:
d = tf.Variable(tf.random_uniform(shape = (3, 2)))

print(d)

<tf.Variable 'Variable_3:0' shape=(3, 2) dtype=float32_ref>


### 2.3 tf.placeholder()

這個 tensor 比較特殊，它的用法是**先產生 tensor 但是不給值，之後再給**，用法如下：

In [5]:
a = tf.placeholder(tf.float32, shape = (3, 2))

print(a)

Tensor("Placeholder:0", shape=(3, 2), dtype=float32)


如此產生了一個 shape 為 (3, 2) 但是還沒有任何數字的 tensor，甚至可以產生連 shape 的某幾維都不明確的 tensor：

In [6]:
b = tf.placeholder(tf.float32, shape = (None, 2))

print(b)

Tensor("Placeholder_1:0", shape=(?, 2), dtype=float32)


這代表這個 tensor 之後可以放入 shape 為 (n, 2) 的矩陣，這裡 n 可以是大於 0 的任意正整數，至於如何給值會在下一個小節探討

## 3. 獲取 tensor 的值

前一節介紹了三種 tensor，可以注意到 tensor 的值沒辦法透過 print() 來查看，那是因為 Tensorflow 有提供自己的一套方法來讓使用者獲取 tensor 的值

### 3.1 tf.Session()

在 Tensorflow 中必須使用 tf.Session() 來獲得 tensor 的內容，具體用法如下：

In [7]:
a = tf.constant(0)
b = tf.constant([1, 2, 3])
c = tf.constant([[2, 4], [6, 8], [10, 12]])

sess = tf.Session()

a_value = sess.run(a)
b_value = sess.run(b)
c_value = sess.run(c)

print(a_value)
print(b_value)
print(c_value)

0
[1 2 3]
[[ 2  4]
 [ 6  8]
 [10 12]]


上面程式碼可以看到，先用 tf.Session() 產生 sess，接著再用 sess.run() 來產生 tensor 的值

接著是察看 tf.Variable() 的方法：

In [8]:
a = tf.Variable([1, 2, 3])
b = tf.Variable(tf.random_uniform(shape = (3, 2)))

sess = tf.Session()

# 非常重要 !
sess.run(tf.global_variables_initializer())

a_value = sess.run(a)
b_value = sess.run(b)

print(a_value)
print(b_value)

[1 2 3]
[[ 0.95073557  0.71619725]
 [ 0.30778825  0.95408607]
 [ 0.9181844   0.03299463]]


tf.Variable() 的察看方法和 tf.constant() 差不多，但是它多了一行 `sess.run(tf.global_variables_initializer())`，如果少了這一行的話程式會發生錯誤，可以注意到由於 b 是使用隨機附值的方式，所以印出來的數值為亂數

接著是察看 tf.placeholder() 的方法：

In [9]:
a = tf.placeholder(tf.int32, shape = (2, 2))
b = tf.placeholder(tf.int32, shape = (None, 2))

sess = tf.Session()

a_value_1 = sess.run(a, feed_dict = {a: [[1,2],[3,4]]})
a_value_2 = sess.run(a, feed_dict = {a: [[5,6],[7,8]]})

# shape = (2, 2)
b_value_1 = sess.run(b, feed_dict = {b: [[1,2],[3,4]]})

# shape = (3, 2)
b_value_2 = sess.run(b, feed_dict = {b: [[5,6],[7,8], [9, 10]]})

print(a_value_1)
print(a_value_2)

print(b_value_1)
print(b_value_2)

[[1 2]
 [3 4]]
[[5 6]
 [7 8]]
[[1 2]
 [3 4]]
[[ 5  6]
 [ 7  8]
 [ 9 10]]


從上面程式碼可以到，透過在 sess.run() 中加入 feed_dict 的參數來給 tf.placeholder() 數值，同時也可以注意一下由於 b 一開始 shape 為 (None, 2) 所以可以給它 shape 為 (2, 2) 或 (3, 2) 的矩陣

## 4. 矩陣運算

### 4.1 線性計算

對於深度學習有初步認識的話，可以知道神經網路每層在經過激活 (activate) 之前會有一個矩陣運算：

$Output = X \times Weight + Bias$

這條數學式中用到了矩陣乘法跟矩陣加法，而 Tensorflow 中可以使用 tf.matmul() 做矩陣乘法，矩陣加法則可以用 + 來做，具體程式碼如下：

In [10]:
X = tf.constant([[1, 2, 3]])
Weight = tf.Variable([[1, -2], [3, -4], [5, -6]])
Bias = tf.Variable([1, 2])

Output = tf.matmul(X, Weight) + Bias

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print(sess.run(Output))

[[ 23 -26]]


上面的程式碼可以用數學表示如下：

$
Output = 
\begin{bmatrix}
    1 & 2 & 3
\end{bmatrix}
\times
\begin{bmatrix}
    1 & -2 \\
    3 & -4 \\
    5 & -6
\end{bmatrix}
+
\begin{bmatrix}
    1 & 2
\end{bmatrix}
$

而計算出來的結果就是 [23, -26]

### 4.2 激活函數

神經網路每層線性計算後會經過激活的階段，Tensorflow 也有提供對應的方法來直接計算激活後的結果，將上一小節程式碼加入激活函數：

In [11]:
X = tf.constant([[1, 2, 3]], dtype = tf.float32)
Weight = tf.Variable([[1, -2], [3, -4], [5, -6]], dtype = tf.float32)
Bias = tf.Variable([1, 2], dtype = tf.float32)

Output = tf.matmul(X, Weight) + Bias

# 使用 Relu 激活函數
Activate = tf.nn.relu(Output)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print(sess.run(Activate))

[[ 23.   0.]]


上面的程式碼使用了 tf.nn.relu() 來做為激活函數，而 relu 會將所有小於 0 的值自動設為 0，大於 0 的值則不變，所以輸出結果從 [23, -26] 變成 [23, 0]，除了 relu 以外還有兩個常見的激活函數，可以試著替換 tf.nn.relu() 並觀察輸出：

- Sigmoid：tf.nn.sigmoid()
- Tanh：tf.nn.tanh()